<a href="https://colab.research.google.com/github/KusalaniR/MedGen.AI/blob/main/notebooks/chatbot_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
from google.api_core.exceptions import ResourceExhausted
import textwrap


In [16]:


from google.colab import drive
drive.mount("/content/drive")

# Load extracted OCR + Gemini results (RAG source 1)
final_df = pd.read_csv(
    "/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/OCR/extracted_report_results.csv"
)

final_df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,test_name,value,status,language,explanation
0,RBC,437.0,Unknown,si,\nTest: RBC\nValue: 437.0\nStatus: Unknown\n\n...
1,MCV,92.0,Normal,si,\nTest: MCV\nValue: 92.0\nStatus: Normal\n\nEx...
2,MCH,287.0,High,si,\nTest: MCH\nValue: 287.0\nStatus: High\n\nExp...
3,MCH,322.0,High,si,\nTest: MCH\nValue: 322.0\nStatus: High\n\nExp...
4,RDW,8.0,Low,si,\nTest: RDW\nValue: 8.0\nStatus: Low\n\nExplan...


In [17]:
#Load medical Knowledge (RAG source 2)
knowledge_df = pd.read_csv(
    "/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_test_knowledge.csv",
    encoding="latin1"
)

knowledge_df.head()


,test_name,normal_range,unit,low_meaning,high_meaning,simple_explanation_en
0,Hemoglobin,"1215.5 (Female), 13.517.5 (Male)",g/dL,May indicate low oxygen-carrying capacity,May indicate dehydration,Hemoglobin is a protein in red blood cells tha...
1,Hematocrit,"3646% (Female), 4153% (Male)",%,May indicate anemia,May indicate dehydration,Hematocrit shows the percentage of red blood c...
2,WBC Count,4.010.0,K/uL,May reduce ability to fight infections,May indicate infection or inflammation,White blood cells help your body fight infecti...
3,Red Blood Cells,"4.25.4 (Female), 4.76.1 (Male)",m/uL,May reduce oxygen delivery,May thicken blood,Red blood cells carry oxygen from your lungs t...
4,Platelet Count,150400,K/uL,May increase bleeding risk,May increase clotting risk,Platelets help your blood to clot and stop ble...


In [18]:
# -------------------------------
# Gemini setup (Chatbot module)
# -------------------------------
import google.generativeai as genai
from google.colab import userdata

# Configure API key
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

# Create Gemini model instance
model = genai.GenerativeModel("models/gemini-flash-lite-latest")

In [19]:
def fallback_chatbot_response(user_question, final_df):
    """
    Rule-based fallback when Gemini quota is exceeded.
    """
    for _, row in final_df.iterrows():
        if row["test_name"].lower() in user_question.lower():
            return (
                f"{row['test_name']} result is {row['value']} "
                f"and the status is {row['status']}. "
                "Please consult a doctor for more details."
            )
    return "I can only answer questions related to your blood report."


In [20]:
#Prepare chatbot context
#because Tell Gemini: These are THIS USER’S test results. Answer only based on this
# -------------------------------
# Build chatbot medical context
# -------------------------------

# def build_report_context(final_df):
#     """
#     Converts extracted blood test results into
#     a readable medical context for the chatbot.
#     """
#     context = "User Blood Test Summary:\n\n"

#     for _, row in final_df.iterrows():
#         context += (
#             f"- Test: {row['test_name']}\n"
#             f"  Value: {row['value']}\n"
#             f"  Status: {row['status']}\n"
#             f"  Explanation: {row['explanation']}\n\n"
#         )

#     return context

def build_report_context(final_df):
    """
    Converts extracted report into readable text for Gemini (RAG context).
    """
    context = ""
    for _, row in final_df.iterrows():
        context += (
            f"Test: {row['test_name']}, "
            f"Value: {row['value']}, "
            f"Status: {row['status']}.\n"
        )
    return context



In [21]:
#Create chatbot prompt (RAG logic)
# def chatbot_prompt(user_question, report_context):
#     """
#     Builds a safe medical chatbot prompt using RAG.
#     """
#     return f"""
# You are a medical report assistant.

# You MUST answer ONLY using the user's blood test report below.
# If the question is unrelated, politely say you cannot answer.

# User Report:
# {report_context}

# User Question:
# {user_question}

# Rules:
# - No diagnosis
# - No medicines
# - Simple, friendly language
# - Educational only
# """

def chatbot_prompt(user_question, report_context):
    """
    Creates a safe, grounded prompt for Gemini chatbot.
    """
    return f"""
You are a medical assistant chatbot.

Below is the patient's blood test report:
{report_context}

User Question:
{user_question}

Rules:
- Answer ONLY using the report
- Do NOT diagnose
- Do NOT suggest medicine
- Use simple language
- Be reassuring
"""


In [24]:
#Gemini chatbot function
# def chatbot_response(user_question, final_df):
#     """
#     Generates chatbot response using report context.
#     Falls back to rule-based response if Gemini fails.
#     """

#     report_context = build_report_context(final_df)
#     prompt = chatbot_prompt(user_question, report_context)

#     try:
#         response = model.generate_content(prompt)
#         return response.text

#     except Exception as e:
#         # Fallback (VERY important for grading)
#         return (
#             "I am unable to use AI right now, but based on your report, "
#             "please review the test values and their status. "
#             "This explanation is educational only."
#         )

def chatbot_response(user_question, final_df):
    """
    Main chatbot function with safe Gemini + fallback handling.
    """
    report_context = build_report_context(final_df)
    prompt = chatbot_prompt(user_question, report_context)

    try:
        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        # Gemini quota / API failure → fallback
        print("⚠ Gemini unavailable, using fallback:", e)
        return fallback_chatbot_response(user_question, final_df)



In [25]:
#Test the chatbot
# Example user questions
print(chatbot_response("Why is my MCH high?", final_df))
print(chatbot_response("Is my cholesterol normal?", final_df))
print(chatbot_response("What does RDW mean?", final_df))


⚠ Gemini unavailable, using fallback: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 16.917336775s.
MCH result is 287.0 and the status is High. Please consult a doctor for more details.


⚠ Gemini unavailable, using fallback: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 16.458428941s.
Cholesterol result is 194.0 and the status is Normal. Please consult a doctor for more details.


⚠ Gemini unavailable, using fallback: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 16.092114839s.
RDW result is 8.0 and the status is Low. Please consult a doctor for more details.
